In [2]:
import Pkg; 
Pkg.add("GLMNet") 
Pkg.add("DataFrames")
Pkg.add("RData")
PKg.add("Random")
using GLMNet
using DataFrames
using Random

   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`


LoadError: UndefVarError: `PKg` not defined

In [9]:
using CSV, DataFrames

#---Loading Data------#
#---------------------#
q3 = CSV.File("../../data/wage2015_subsample_inference.csv") |> DataFrame

n = size(q3, 1)
nvar = size(q3, 2)
println(nvar)
q3.id = 1:n  # id variable
println(n)
println(nvar)

21
5150
21


In [10]:
using GLMNet
using Random

#--Split----#
Random.seed!(1234)
random = randperm(n)[1:floor(Int, n * 0.8)]  # 80/20 split

#Training set
train = q3[random, :]
index_train =  q3.id[random]   # Original indexes for Training set
ntrain = size(train, 1)

#Testing set
test = q3[setdiff(1:n, random), :]
index_test =  q3.id[setdiff(1:n, random)]   # Original indexes for Testing set
ntest = size(test, 1)

println(ntest)

1030


In [11]:
#---(2) Lambda range------#
#-------------------------#
lambdas = 0.1:0.1:0.5

#---(3) Partition------#
#----------------------#
k = 5
obsfold = ntrain ÷ k  # 824 obs. per fold
cutoff = [obsfold * i for i in 1:k]

#--Folds----#
index_f1 = index_train[1:cutoff[1]]
index_f2 = index_train[(cutoff[1]+1):cutoff[2]]
index_f3 = index_train[(cutoff[2]+1):cutoff[3]]
index_f4 = index_train[(cutoff[3]+1):cutoff[4]]
index_f5 = index_train[(cutoff[4]+1):cutoff[5]]

824-element Vector{Int64}:
  730
 3436
 2378
 4421
  907
 2786
 3731
  336
 3340
 1733
 1494
 5088
  811
    ⋮
 4102
 1151
 4317
  888
  274
  249
 1209
 3793
 3654
 3679
 4768
 3146

In [12]:
#---(4) Lasso function------#   
#---------------------------#
import Pkg;
Pkg.add("Lasso")
using Lasso
using GLMNet

#---(5) CV loop------#
#--------------------#
folds = hcat(index_f1, index_f2, index_f3, index_f4, index_f5)
MSE_mat = fill(NaN, 5, length(lambdas))
print(size(folds)) 

p = 0.1
i = 1
index_training = folds[:, setdiff(1:k, i)]
index_validation = folds[:, i]

x = q3[!, Not(["wage", "lwage", "id"])]
y = q3.wage
print(size(x))
print(size(y))

model = glmnet(Matrix(x), Matrix(y), alpha=1, lambda=0.1) #issue

   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`


(824, 5)(5150, 19)(5150,)[9.615384615384615, 48.07692307692308, 11.057692307692308, 13.942307692307692, 28.846153846153847, 11.73076923076923, 19.23076923076923, 19.23076923076923, 12.0, 19.23076923076923, 17.307692307692307, 12.01923076923077, 12.01923076923077, 13.461538461538462, 16.346153846153847, 27.884615384615383, 21.6, 8.653846153846153, 19.23076923076923, 13.186813186813186, 10.683760683760683, 11.538461538461538, 17.78846153846154, 19.23076923076923, 16.304347826086957, 19.23076923076923, 14.423076923076923, 12.0, 16.826923076923077, 19.67032967032967, 42.30769230769231, 13.942307692307692, 10.096153846153847, 14.053254437869823, 11.965811965811966, 11.538461538461538, 24.03846153846154, 16.272189349112427, 11.538461538461538, 41.20879120879121, 21.634615384615383, 11.217948717948717, 6.730769230769231, 10.096153846153847, 12.01923076923077, 14.022435897435898, 28.846153846153847, 7.211538461538462, 11.778846153846153, 26.923076923076923, 12.937062937062937, 13.0769230769230

Excessive output truncated after 1054780 bytes.

LoadError: MethodError: no method matching (Matrix)(::Vector{Float64})

[0mClosest candidates are:
[0m  (Array{T, N} where T)(::AbstractArray{S, N}) where {S, N}
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:498[24m[39m
[0m  (Matrix)([91m::UndefInitializer[39m, [91m::Integer[39m, [91m::Integer[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mbaseext.jl:33[24m[39m
[0m  (Matrix)([91m::LinearAlgebra.LU{T, LinearAlgebra.Tridiagonal{T, V}}[39m) where {T, V}
[0m[90m   @[39m [36mLinearAlgebra[39m [90mC:\Users\ALBERTO TRELLES\AppData\Local\Programs\Julia-1.10.2\share\julia\stdlib\v1.10\LinearAlgebra\src\[39m[90m[4mlu.jl:755[24m[39m
[0m  ...
